# Multi-Layer Perceptron (MLP) com Iris Dataset

## Objetivo
Implementação completa de uma rede MLP para classificação multiclasse utilizando o dataset Iris, demonstrando os conceitos fundamentais de redes neurais feedforward.

## 1. Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Configuração de visualização
plt.style.use('seaborn-v0_8')
np.random.seed(42)
tf.random.set_seed(42)

## 2. Carregamento e Análise Exploratória dos Dados

In [ ]:
# Carregamento do dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Conversão para DataFrame para análise
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['species'] = df['target'].map({i: name for i, name in enumerate(target_names)})

print("Informações do Dataset:")
print(f"Shape: {X.shape}")
print(f"Features: {feature_names}")
print(f"Classes: {target_names}")
print("\nPrimeiras 5 amostras:")
print(df.head())
print("\nEstatísticas descritivas:")
print(df.describe())

### Visualização da Distribuição dos Dados

In [ ]:
# Pairplot para visualizar relações entre features
plt.figure(figsize=(12, 10))
sns.pairplot(df, hue='species', diag_kind='kde', markers=['o', 's', 'D'])
plt.suptitle('Distribuição das Features por Espécie', y=1.02)
plt.show()

# Boxplots para cada feature
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for idx, feature in enumerate(feature_names):
    df.boxplot(column=feature, by='species', ax=axes[idx])
    axes[idx].set_title(f'Distribuição de {feature}')
    axes[idx].set_xlabel('Espécie')
    axes[idx].set_ylabel(feature)

plt.tight_layout()
plt.show()

## 3. Pré-processamento dos Dados

In [ ]:
# Normalização das features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encoding dos labels
y_encoded = to_categorical(y, num_classes=3)

# Divisão em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y
)

# Divisão adicional para conjunto de validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train.argmax(axis=1)
)

print(f"Conjunto de treino: {X_train.shape[0]} amostras")
print(f"Conjunto de validação: {X_val.shape[0]} amostras")
print(f"Conjunto de teste: {X_test.shape[0]} amostras")

## 4. Arquitetura do MLP

### Modelo Base com Configuração Otimizada

In [ ]:
def create_mlp_model(input_dim, hidden_layers, activation='relu', dropout_rate=0.3):
    """
    Cria um modelo MLP configurável.

    Args:
        input_dim: Dimensão da entrada
        hidden_layers: Lista com número de neurônios por camada oculta
        activation: Função de ativação para camadas ocultas
        dropout_rate: Taxa de dropout
    """
    model = Sequential()

    # Primeira camada oculta
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation=activation))
    model.add(Dropout(dropout_rate))

    # Camadas ocultas adicionais
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout_rate))

    # Camada de saída
    model.add(Dense(3, activation='softmax'))

    return model

# Criação do modelo
model = create_mlp_model(
    input_dim=4,
    hidden_layers=[64, 32, 16],
    activation='relu',
    dropout_rate=0.3
)

# Compilação
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumo da arquitetura
model.summary()

### Visualização da Arquitetura

In [ ]:
# Contagem de parâmetros por camada
total_params = 0
trainable_params = 0
non_trainable_params = 0

print("\nDetalhamento dos parâmetros por camada:\n")
for layer in model.layers:
    layer_params = layer.count_params()
    if hasattr(layer, 'trainable_weights'):
        trainable = sum([tf.size(w).numpy() for w in layer.trainable_weights])
        non_trainable = layer_params - trainable
    else:
        trainable = layer_params
        non_trainable = 0

    print(f"{layer.name:20} | Parâmetros: {layer_params:7,} | Treináveis: {trainable:7,}")
    total_params += layer_params
    trainable_params += trainable
    non_trainable_params += non_trainable

print(f"\nTotal de parâmetros: {total_params:,}")
print(f"Parâmetros treináveis: {trainable_params:,}")
print(f"Parâmetros não treináveis: {non_trainable_params:,}")

## 5. Treinamento do Modelo

In [ ]:
# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_mlp_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=0
)

# Treinamento
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

## 6. Visualização do Histórico de Treinamento

In [ ]:
# Extração do histórico
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs_range = range(1, len(train_loss) + 1)

# Plotagem
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(epochs_range, train_loss, 'b-', label='Loss de Treino', linewidth=2)
axes[0].plot(epochs_range, val_loss, 'r-', label='Loss de Validação', linewidth=2)
axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Loss')
axes[0].set_title('Evolução da Loss durante o Treinamento')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Acurácia
axes[1].plot(epochs_range, train_acc, 'b-', label='Acurácia de Treino', linewidth=2)
axes[1].plot(epochs_range, val_acc, 'r-', label='Acurácia de Validação', linewidth=2)
axes[1].set_xlabel('Épocas')
axes[1].set_ylabel('Acurácia')
axes[1].set_title('Evolução da Acurácia durante o Treinamento')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Métricas finais
final_epoch = len(train_loss)
print(f"\nMétricas na época {final_epoch}:")
print(f"Loss de Treino: {train_loss[-1]:.4f}")
print(f"Loss de Validação: {val_loss[-1]:.4f}")
print(f"Acurácia de Treino: {train_acc[-1]:.4f}")
print(f"Acurácia de Validação: {val_acc[-1]:.4f}")

## 7. Avaliação no Conjunto de Teste

In [ ]:
# Avaliação
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nDesempenho no Conjunto de Teste:")
print(f"Loss: {test_loss:.4f}")
print(f"Acurácia: {test_acc:.4f}")

# Predições
y_pred_proba = model.predict(X_test)
y_pred = y_pred_proba.argmax(axis=1)
y_true = y_test.argmax(axis=1)

## 8. Matriz de Confusão e Relatório de Classificação

In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Matriz de Confusão - MLP')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.show()

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=target_names))

## 9. Análise de Confiança das Predições

In [ ]:
# Análise das probabilidades preditas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição das probabilidades máximas
max_probs = y_pred_proba.max(axis=1)
axes[0].hist(max_probs, bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Probabilidade Máxima')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição da Confiança das Predições')
axes[0].grid(True, alpha=0.3)

# Heatmap das probabilidades por amostra
im = axes[1].imshow(y_pred_proba.T, aspect='auto', cmap='viridis')
axes[1].set_yticks(range(3))
axes[1].set_yticklabels(target_names)
axes[1].set_xlabel('Amostras de Teste')
axes[1].set_ylabel('Classes')
axes[1].set_title('Probabilidades Preditas por Amostra')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

# Análise de amostras com baixa confiança
low_confidence_threshold = 0.7
low_confidence_samples = np.where(max_probs < low_confidence_threshold)[0]
print(f"\nAmostras com confiança < {low_confidence_threshold}: {len(low_confidence_samples)}")

if len(low_confidence_samples) > 0:
    print("\nDetalhes das amostras de baixa confiança:")
    for idx in low_confidence_samples[:5]:  # Mostrar apenas as primeiras 5
        print(f"\nAmostra {idx}:")
        print(f"  Classe verdadeira: {target_names[y_true[idx]]}")
        print(f"  Classe predita: {target_names[y_pred[idx]]}")
        print(f"  Probabilidades: {dict(zip(target_names, y_pred_proba[idx]))}")

## 10. Experimentação com Diferentes Funções de Ativação

In [ ]:
# Comparação de funções de ativação
activation_functions = ['relu', 'tanh', 'sigmoid', 'elu']
results = {}

for activation in activation_functions:
    print(f"\nTreinando modelo com ativação: {activation}")

    # Criar e compilar modelo
    model_temp = create_mlp_model(
        input_dim=4,
        hidden_layers=[64, 32],
        activation=activation,
        dropout_rate=0.2
    )

    model_temp.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Treinar (com menos épocas para comparação rápida)
    history_temp = model_temp.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        verbose=0
    )

    # Avaliar
    test_loss, test_acc = model_temp.evaluate(X_test, y_test, verbose=0)

    results[activation] = {
        'history': history_temp.history,
        'test_acc': test_acc,
        'test_loss': test_loss
    }

    print(f"  Acurácia no teste: {test_acc:.4f}")

### Visualização Comparativa

In [ ]:
# Plotagem comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for activation, result in results.items():
    epochs = range(1, len(result['history']['loss']) + 1)

    # Loss de validação
    axes[0].plot(epochs, result['history']['val_loss'], label=activation)

    # Acurácia de validação
    axes[1].plot(epochs, result['history']['val_accuracy'], label=activation)

axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Loss de Validação')
axes[0].set_title('Comparação de Loss por Função de Ativação')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Épocas')
axes[1].set_ylabel('Acurácia de Validação')
axes[1].set_title('Comparação de Acurácia por Função de Ativação')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabela de resultados finais
print("\nResultados finais por função de ativação:")
print("-" * 40)
print(f"{'Ativação':>10} | {'Acurácia':>10} | {'Loss':>10}")
print("-" * 40)
for activation, result in results.items():
    print(f"{activation:>10} | {result['test_acc']:>10.4f} | {result['test_loss']:>10.4f}")

## 11. Análise de Sensibilidade dos Pesos

In [ ]:
# Análise dos pesos da primeira camada
first_layer_weights = model.layers[0].get_weights()[0]  # Pesos
first_layer_bias = model.layers[0].get_weights()[1]     # Bias

# Visualização dos pesos
plt.figure(figsize=(10, 8))
plt.imshow(first_layer_weights.T, aspect='auto', cmap='coolwarm',
           vmin=-np.abs(first_layer_weights).max(),
           vmax=np.abs(first_layer_weights).max())
plt.colorbar(label='Valor do Peso')
plt.xlabel('Features de Entrada')
plt.ylabel('Neurônios da Primeira Camada Oculta')
plt.title('Mapa de Calor dos Pesos da Primeira Camada')
plt.xticks(range(4), feature_names, rotation=45)
plt.tight_layout()
plt.show()

# Análise da importância das features baseada nos pesos
feature_importance = np.abs(first_layer_weights).mean(axis=1)
feature_importance_normalized = feature_importance / feature_importance.sum()

plt.figure(figsize=(8, 6))
bars = plt.bar(feature_names, feature_importance_normalized)
plt.xlabel('Features')
plt.ylabel('Importância Relativa')
plt.title('Importância das Features Baseada nos Pesos da Primeira Camada')
plt.xticks(rotation=45)

# Adicionar valores nas barras
for bar, importance in zip(bars, feature_importance_normalized):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{importance:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 12. Desafio: Problema de Regressão com MLP

### Implementação de um MLP para Regressão

In [ ]:
# Geração de dados sintéticos para regressão
np.random.seed(42)
X_reg = np.linspace(-3, 3, 300).reshape(-1, 1)
y_reg = X_reg**2 + 0.5 * X_reg**3 + 2 * np.sin(2 * X_reg) + np.random.normal(0, 0.5, X_reg.shape)
y_reg = y_reg.ravel()

# Divisão dos dados
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Normalização
scaler_X_reg = StandardScaler()
X_train_reg_scaled = scaler_X_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_X_reg.transform(X_test_reg)

# Modelo de regressão
model_regression = Sequential([
    Dense(64, activation='relu', input_dim=1),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Sem ativação para regressão
])

model_regression.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

# Treinamento
history_reg = model_regression.fit(
    X_train_reg_scaled, y_train_reg,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=0
)

# Avaliação
test_loss_reg, test_mae_reg = model_regression.evaluate(
    X_test_reg_scaled, y_test_reg, verbose=0
)
print(f"\nResultados da Regressão:")
print(f"MSE no teste: {test_loss_reg:.4f}")
print(f"MAE no teste: {test_mae_reg:.4f}")

### Visualização dos Resultados da Regressão

In [ ]:
# Predições
X_plot = np.linspace(-3, 3, 500).reshape(-1, 1)
X_plot_scaled = scaler_X_reg.transform(X_plot)
y_pred_reg = model_regression.predict(X_plot_scaled)

# Plotagem
plt.figure(figsize=(12, 6))

# Dados e predições
plt.subplot(1, 2, 1)
plt.scatter(X_reg, y_reg, alpha=0.5, s=20, label='Dados reais')
plt.plot(X_plot, y_pred_reg, 'r-', linewidth=2, label='Predição MLP')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('MLP para Regressão Não-Linear')
plt.legend()
plt.grid(True, alpha=0.3)

# Histórico de treinamento
plt.subplot(1, 2, 2)
plt.plot(history_reg.history['loss'], label='Loss de Treino')
plt.plot(history_reg.history['val_loss'], label='Loss de Validação')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.title('Evolução do Erro durante o Treinamento')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Conclusões

### Principais Observações:

1. **Arquitetura MLP**: Demonstramos a implementação completa de um MLP para classificação multiclasse
2. **Funções de Ativação**: ReLU mostrou melhor desempenho para este problema
3. **Regularização**: Dropout foi eficaz na prevenção de overfitting
4. **Versatilidade**: O MLP se adapta tanto para classificação quanto regressão

### Conceitos Demonstrados:
- Propagação forward através de camadas densas
- Importância das funções de ativação não-lineares
- Técnicas de regularização (Dropout)
- Análise de pesos e importância de features
- Adaptação para problemas de regressão

### Teorema da Aproximação Universal:
Os resultados confirmam a capacidade do MLP de aproximar funções complexas, tanto para classificação quanto para regressão não-linear.